In [ ]:
import robotic as ry
import numpy as np
import matplotlib.pyplot as plt
import random
import time

environment="tunnel_tool_env.g"
config = ry.Config()
config.addFile(environment)
config.view()

config_2 = ry.Config()
config_2.copy(config)

config_2.view()

#Initialize Action/Path Tree
action_tree=[]

#Collect Frame States (it goes incrementally according to t. First frame is initial Config)
frame_list=[]

#Goal_Pose and Contact_Point list
final_pose_list= []
contact_point_list= []

box = config.frame("box")
tool = config.frame("stick")

path_list = [[config.getJointState()]]
object_list = [[*box.getPosition(), *box.getQuaternion()]]
tool_list = [[*tool.getPosition(), *tool.getQuaternion()]]

#Action Hyperparameters
#delta=0.2 #Duration of the action (Such as distance between nodes)
delta=0.2 #Duration of the action (Such as distance between nodes)
phase=30 #Komo stepsPerPhase
trial=8 #Number of action trials
max_iter=40 #Max iteration number of main loop (=WP count) (=len(path))
rrt_extend_length=0.1 #For RRT (not used right now)
threshold= 0.08 # Threshold of goal pose

goal_pos= np.array([-0.7, -0.4, 0.69001, 1, 0, 0, 0])

#Define RRT
def RRT(qF,rrt_extend_length=0.04):
    C2 = ry.Config()
    C2.addFile("tunnel_tool_actuated_env.g")
    #C2.view()
    q0 = C2.getJointState()
    print("Start State: ", q0," Final State: ", qF)
    # Create an RRT path finder
    rrt = ry.PathFinder()
    rrt.setProblem(C2, [q0], [qF])

    isFeas = False
    path = None

    i=0
    # Find an RRT path
    while i!=10 and (not isFeas):
        rrt = ry.PathFinder()
        rrt.setProblem(C2, [q0], [qF])
        ret = rrt.solve()
        isFeas = ret.feasible
        path = ret.x
        i+=1

    ## Smooth the path (optimization) ##

    #Create a KOMO for post-optimization with acceleration constraints
    num_time_steps = len(path)

    komo2 = ry.KOMO(C2, num_time_steps, 1, 2, True)
    #Add an acceleration constraint objective to the new KOMO object
    komo2.addControlObjective([], 2, 1e1)
    komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e2])
    #Set the waypoints for the KOMO
    komo2.initWithPath_qOrg(path)

    ret = ry.NLP_Solver() \
        .setProblem(komo2.nlp()) \
        .setOptions(stopTolerance=1e-2, verbose=0) \
        .solve()
    print(ret)

    path = komo2.getPath()

    ## Interpolate the path ##
    new_path = []
    points = path

    for i in range(len(points) - 1):
        current_point = np.array(points[i])
        next_point = np.array(points[i + 1])

        distance = np.linalg.norm(next_point - current_point)

        # Calculate the number of intermediate points needed
        num_intermediate_points = int(distance / rrt_extend_length)

        if num_intermediate_points > 0:
            # Compute the step size for interpolation
            step_size = 1.0 / (num_intermediate_points + 1)

            # Perform linear interpolation between consecutive points
            for j in range(1, num_intermediate_points + 1):
                interpolated_point = current_point + j * step_size * (next_point - current_point)
                new_path.append(interpolated_point.tolist())
        else:
            new_path.append(current_point.tolist())

    #Add the last point
    new_path.append(points[-1])
    print("Length of the Path:", len(new_path))

    for i in range(0, len(new_path)):
        C2.setJointState(new_path[i])
        C2.view()
        time.sleep(0.02)

    return new_path

#### Use Vision/ Point cloud for CP estimation ####
def getCameraView(C, camName, filter = 10, plot = False):
    cam = ry.CameraView(C)
    cam.setCamera(camName)
    rgb, depth = cam.computeImageAndDepth(C)

    # Filtering the object
    mask = np.zeros(rgb[:,:,1].shape)
    red = rgb[:,:,0]
    green = rgb[:,:,1]
    blue = rgb[:,:,2]
    
    (x, y) = mask.shape
    for i in range(0, x):
        for j in range(0, y):
            if green[i][j] > 180 and blue[i][j] > 180 and red[i][j] < 50:
                mask[i][j] = 1
                
    newDepth = np.where(mask, depth, np.nan)
    pcl = ry.depthImage2PointCloud(newDepth, cam.getFxycxy())

    # Removing the other points
    cloud = []
    (x, y) = depth.shape
    for i in range(0, x):
        for j in range(0, y):
            if mask[i][j] and j%filter == 0:
                cloud.append(pcl[i][j])
    cloud = np.asarray(cloud)
    
    if cloud.size!=0:
        #p1 = C.addFrame('pcl_' + str(random.randint(0, 1000)), camName)
        #p1.setPointCloud(cloud, [0,1,1])
        #C.view()
        
        if plot:
            fig = plt.figure()
            fig.add_subplot(1,2,1)
            plt.imshow(rgb)
            fig.add_subplot(1,2,2)
            plt.imshow(depth)
            plt.show()

    return [cam, cloud]

prev_c=[]
def findGraspPoint(C, c):
    global prev_c
    if c.size==0:
        print("Couldn't find point cloud")
        c=prev_c
        
    cam = C.frame("cam_5")
    grasp_point = cam.getPosition() - np.array(c[random.randint(0, (c.shape[0] - 1)), :])
    grasp_point[0] *= -1
    prev_c = c
    return grasp_point

def isContactFeasible(C, t, contact_point):
    S = ry.Skeleton()
    S.enableAccumulatedCollisions(True)
    S.addEntry([t, t], ry.SY.touch,  ["gripper", contact_point])
    komo = S.getKomo_path(C, stepsPerPhase=30, accScale=1e0, lenScale=1e-2, homingScale=1e-1, collScale=1e3)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return ret.feasible

def addMarker(C, pos, name, quat = [1, 0, 0, 0], obj= "box"):
    marker = C.addFrame(name, parent= obj) \
              .setPosition([*pos[:2], pos[2]]) \
              .setQuaternion(quat)
    return 

def addPoint(C, pos, name, parent, color):
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setPosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

def convertCPrelative(C,new_name="rel_contact_point",name="contact_point", obj= "box"):
    relativeCP= config.getFrame(name)
    rel_CP_pos=relativeCP.getRelativePosition()
    relCP = C.addFrame(new_name, parent= obj) \
              .setRelativePosition(rel_CP_pos) 

#Generate Contact point
def CPgeneration(C, obj= "box"):
    [p1, c] = getCameraView(C, "cam_5", 10, False)
    
    if c.size==0:
        return None
    grasp_point=findGraspPoint(C, c)
    print(f"Contact Point= {grasp_point}")
    marker = addMarker(C, grasp_point, "contact_point",obj=obj)
    convertCPrelative(C,"rel_contact_point", obj=obj)
    #marker = addMarker(C, grasp_point, "contact_point",  [0.7038453156522361, 0.0, 0.0, 0.7103532724176078])
    itr=0
    while itr!=10 and (not isContactFeasible(C, 1, "rel_contact_point")):
        itr+=1
        C.delFrame("contact_point")
        grasp_point = findGraspPoint(C, c)
        addMarker(C, grasp_point, "contact_point",obj=obj)
        convertCPrelative(C,"rel_contact_point","contact_point",obj=obj)
        #marker = addMarker(C, grasp_point, "contact_point", 0.1, [0.7038453156522361, 0.0, 0.0, 0.7103532724176078])
        print(f"This Contact is feasible (found in {itr} iteration)")
    addPoint(config_2, grasp_point, "grasp_point_"+str(random.randint(0, 100000)), "box", [1, 1, 0])

#Pick Action
def pick(C, contact_point,final_pose,t=1):
    
    addPoint(config_2, C.frame(final_pose).getPosition(), "grasp_point_"+str(random.randint(0, 100000)), "world", [1, 0, 0])
    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "box"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper","box"])
    skeleton.addEntry([t+delta, t+delta], ry.SY.poseEq, ["box", final_pose])
    #skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(C, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-2, collScale=1e1)
    komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    
    return komo,ret

#Push Actions
def push_stick(C, contact_point,final_pose,t=1):
    
    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "stick"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper", "stick"])  #Check this part for optimal motion

    skeleton.addEntry([t+delta, -1], ry.SY.stable,  ["stick","box"])  
    skeleton.addEntry([t+delta, -1], ry.SY.touch,  ["stick","box"])
    skeleton.addEntry([t+2*delta, t+2*delta], ry.SY.poseEq, ["box", final_pose])
    skeleton.enableAccumulatedCollisions(True)
    
    komo = skeleton.getKomo_path(C, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-3, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

def push_ball(contact_point,final_pose,t=1):

    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "ball"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper", "ball"])  #Check this part for optimal motion

    skeleton.addEntry([t+delta, -1], ry.SY.stable,  ["ball","box"])  
    skeleton.addEntry([t+delta, -1], ry.SY.touch,  ["ball","box"])
    skeleton.addEntry([t+2*delta, t+2*delta], ry.SY.poseEq, ["box", final_pose])
    skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-3, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

def push_cube(contact_point,final_pose,t=1):

    skeleton = ry.Skeleton()
    skeleton.addEntry([t, -1], ry.SY.stable,  ["gripper", "cube"])
    skeleton.addEntry([t, -1], ry.SY.touch,  ["gripper", "cube"])  #Check this part for optimal motion

    skeleton.addEntry([t+delta, -1], ry.SY.stable,  ["cube","box"])  
    skeleton.addEntry([t+delta, -1], ry.SY.touch,  ["cube","box"])
    skeleton.addEntry([t+2*delta, t+2*delta], ry.SY.poseEq, ["box", final_pose])
    skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_path(config, stepsPerPhase=phase, accScale=1e0, lenScale=1e-2, homingScale=1e-3, collScale=1e1)
    #komo.addObjective([t, -1], ry.FS.positionDiff, ["gripper", contact_point], ry.OT.sos, target = [0.0], scale=[1e2])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    return komo,ret

#WP feasibility check
def tryWP(gen_wp,i):
    print(f"Generated WP is: {gen_wp}")

    #Create as frame
    wpx=config.addFrame(name=f'wp{i}')
    wpx.setPosition(gen_wp[0:3]) #Generate WP Position
    wpx.setQuaternion(gen_wp[3:])   
    addPoint(config_2, config.frame("box").getPosition(), "point"+str(random.randint(0,10000)), "world", [1, 0, 0])
    return gen_wp    

def Point_Cloud_Flag(C):
    [p1, c] = getCameraView(C, "cam_5", 10, False)
    if c.size==0:
        return False
    else:
        return True


In [ ]:
path= RRT(goal_pos,rrt_extend_length)

In [3]:
i=0
CPgeneration(config, obj= "box") #Initial CP generator, it uses path[0] (first WP)
#convertCPrelative(config,"contact_point", obj= "box")
gen_cp= "rel_contact_point" #This will come from CP creator
gen_wp=np.zeros(np.shape(goal_pos))
counter=0
while i!=max_iter and not np.linalg.norm(gen_wp - goal_pos) <threshold:
    print(f'Step: {i+1}')
    fail=0
    flag=Point_Cloud_Flag(config)
    for n in range(trial):
        #gen_wp = WPgeneration() #Such as Extent function of RRT
        gen_wp=path[i]
        tryWP(gen_wp,i)
        #CPgeneration(config, obj= "stick")
        #print(flag)
        
        if flag==True:
            counter+=1
        
        if counter<=3:
            #Test Waypoint:#
            #Parameter Inıtialization
            end_pose= f'wp{i}'
            contact_point= gen_cp
            delta=0.5    
            t = 0.4 if n >= 2 else 1 if i == 0 else 0
            # Solve push_stick Action with KOMO
            komo, ret = push_stick(config, contact_point, end_pose, t)
            waypoints = komo.getPath_qAll()
            for j in range(0, len(waypoints)):
                config.setFrameState(komo.getFrameState(j))
                tool_list.append([*tool.getPosition(), *tool.getQuaternion()])
                addPoint(config_2, config.frame("stick").getPosition(), "grasp_point_"+str(random.randint(0, 100000)), "world", [1, 0, 0])
                config_2.view()
                
        else:
            delta=0.4
            
            if counter<=5:
                CPgeneration(config, obj= "box") #This will become actual CP generator
                convertCPrelative(config,"contact_point", obj= "box")
                gen_cp= "rel_contact_point" 

            elif n >=2:
                CPgeneration(config, obj= "box") #This will become actual CP generator
                convertCPrelative(config,"contact_point", obj= "box")
                gen_cp= "rel_contact_point" 
            
            #Test Waypoint:#
            #Parameter Inıtialization
            end_pose= f'wp{i}'
            contact_point= gen_cp

            
            if i==0:    #There is a problem when t is not always equal to 1
                t=1
                #Solve Pick Action with KOMO
                komo,ret = pick(config, contact_point,end_pose,t)
            elif n>=2:   
                t=0.4
                #Solve push Action with KOMO
                komo,ret = pick(config, contact_point,end_pose,t)
            else:
                t=0
                #Solve push Action with KOMO
                komo,ret = pick(config, contact_point,end_pose,t)

        if ret.eq<=0.8:

            config.view()

            print("Action is feasible") 
            path_list.append(komo.getPath())
            object_list.append([*box.getPosition(), *box.getQuaternion()])
            tool_list.append([*tool.getPosition(), *tool.getQuaternion()])

            #Set Final Configuration of current action
            waypoints = komo.getPath_qAll()
            Frame_state=komo.getFrameState(len(waypoints)-1)
            frame_list.append(Frame_state)
            final_pose_list.append(gen_wp)
            contact_point_list.append(gen_cp)
            config.setFrameState(Frame_state)
            #addPoint(config, config.frame("stick").getPosition(), "grasp_point_"+str(random.randint(0, 100000)), "world", [1, 0, 0])
            break #Continue to next WP in the path

        else:
            print(f"Infeasible Action, Generate new WP,CP pair: {n}")
            feasibility= False
            fail+=1

    i+=1    
 
    if fail!=(trial):
        continue
    else:
        #Generate new WP list (path) in this case
        #WPgeneration()
        print("New path is calculated")
        print("Try to use smaller 'extend_length' or higher 'phase'. Play with Hyperparameters")
        print("Action Failed")
        break

if np.linalg.norm(gen_wp - goal_pos) <threshold:
    print("TRAJECTORY FOUND!!!!")

# Visualization

In [ ]:
### Visualization ###
bo = config_2.frame("box")
tool = config_2.frame("stick")
config_2.view()

for i, j, k in zip(path_list, object_list, tool_list):
    bo.setPosition(j[0:3])
    bo.setQuaternion(j[3:])
    tool.setPosition(k[0:3])
    tool.setQuaternion(k[3:])
    for l in i:
        config_2.setJointState(l) 
        config_2.view()
        time.sleep(0.2)

In [ ]:
for i in config_2.getFrameNames():
    if "panda" in i or i in ["gripper", "palm", "finger1", "finger2"]:
        config_2.delFrame(i)

config_2.view()